In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import collections
import fuzzywuzzy
from pandas_profiling import ProfileReport
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import  accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from collections import Counter
from imblearn.over_sampling import SMOTE
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [2]:
train = pd.read_csv('train++.csv')


In [3]:
train

,Unnamed: 0,Unnamed: 0.1,review_id,review_text,category,langdetect,stop-stem sastrawi,stop-stem english,hasil_deteksi_bahasa,caps,jumlah_caps,capslock,pakai_tanda_seru,jumlah_tanda_seru,stop_stem_ganda,stop_stem_id,stop_stem_en
0,0,0,0fc651bfdd38a3b4ba19ef9d0d1e92e3,"Penginapannya rapi, bersih, suasana tenang, pe...",1,id,id,id,id,0,0,0,0,0,inap rapi bersih suasana tenang layan ramah ba...,inap rapi bersih suasana tenang layan ramah ba...,penginapannya rapi bersih suasana tenang pelay...
1,1,1,7d8cb2e465c5602e51c1ba8710e8ff16,"wifi tidak sampai kamar, tidak tersedia heater...",0,id,id,en,id,0,0,0,0,0,wifi kamar sedia heater kopi teh dispens sedia...,wifi kamar sedia heater kopi teh dispenser sed...,wifi tidak sampai kamar tidak tersedia heater ...
2,2,2,1f4447c73e5e1aac462d05b0ecbd5a55,"Lantai kamar mandi kotor, bau, dan seperti sdh...",0,id,id,en,id,0,0,0,0,0,lantai kamar mandi kotor bau sdh jdi sarang ny...,lantai kamar mandi kotor bau sdh jdi sarang ny...,lantai kamar mandi kotor bau dan seperti sdh j...
3,3,3,b9aefbfb0debbfff43f8b268c82b43c0,"Wc jorok.. Kasur tidak dibersihkan,, handuk ti...",0,id,id,id,id,0,0,0,0,0,wc jorok kasur bersih handuk ganti,wc jorok kasur bersih handuk ganti,wc jorok kasur tidak dibersihkan handuk tidak ...
4,4,4,e2afef18bd695fae43d4d53b8c547bd4,bocor lagi,0,id,id,id,id,0,0,0,0,0,bocor,bocor,bocor lagi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14851,14851,14851,5ed96732e4f1e1e69dfb8669cf507587,kotor dan tidak ada wifi,0,id,id,id,id,0,0,0,0,0,kotor wifi,kotor wifi,kotor dan tidak ada wifi
14852,14852,14852,b0692bdf4926ee0d58c2d9a0f28b0d62,Kamar tidak bersih,0,id,id,id,id,0,0,0,0,0,kamar bersih,kamar bersih,kamar tidak bersih
14853,14853,14853,5217f5f284066f232d7b3c3bbd3cccd2,kamarnya terlalu kecil beberapa cm antara kasu...,0,id,id,en,id,0,0,0,0,0,kamar cm kasur dind,kamar cm kasur dinding,kamarnya terlalu kecil beberapa cm antara kasu...
14854,14854,14854,31ca5a04ccc0dae4eaafc91c9822bede,Wifinya sangat kurang memadai..,0,id,id,id,id,0,0,0,0,0,wifinya pada,wifinya pada,wifinya sangat kurang memadai


In [4]:
mantab = train[train['langdetect']!='id']

In [5]:
mantab

,Unnamed: 0,Unnamed: 0.1,review_id,review_text,category,langdetect,stop-stem sastrawi,stop-stem english,hasil_deteksi_bahasa,caps,jumlah_caps,capslock,pakai_tanda_seru,jumlah_tanda_seru,stop_stem_ganda,stop_stem_id,stop_stem_en
71,71,71,8b8a416ad10d934589a6824128fcb2f2,having great services and strategic locations ...,1,en,en,en,en,0,0,0,0,0,great servic strateg locat near culinari spot,having great services and strategic locations ...,great servic strateg locat near culinari spot
91,91,91,24e3f9f0e90cf4ccb4dece45d8975476,"the room was comfort, big, and the facilities ...",1,en,en,en,en,0,0,0,0,0,room comfort big facil good strateg place cent...,the room was comfort big and the facilities wa...,room comfort big facil good strateg place cent...
129,129,129,653575979493fbc24b6aa17f128cf98e,clean romm and new building .,1,en,en,en,en,0,0,0,0,0,clean romm new build,clean romm and new building,clean romm new build
199,199,199,16aef4b7249a930deefd2f887080e5ab,"nice room, love it, good food and decdnt price...",1,en,en,en,en,0,0,0,0,0,nice room love good food decdnt price room ser...,nice room love it good food and decdnt price a...,nice room love good food decdnt price room ser...
305,305,305,67f6312614cd51478569dbaae66bf9d1,Although we only stayed 1 night but overall we...,1,en,en,en,en,0,0,0,0,0,although stay 1 night overal satisfi facil ser...,although we only stayed 1 night but overall we...,although stay 1 night overal satisfi facil ser...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14812,14812,14812,d208222686cb24dd93bf3b04c163ff1f,"no phone number, airy cs is slow\n",0,en,en,en,en,0,0,0,0,0,phone number air cs slow,no phone number airy cs is slow,phone number airi cs slow
14821,14821,14821,3ffc99e901756ba338fa98e25d200654,"GUA GOBLOK, GUA JELEK, TANDA TANGAN GUA JELEK",0,en,en,id,en,0,0,1,0,0,gua goblok gua jelek tanda tangan gua jelek,gua goblok gua jelek tanda tangan gua jelek,gua goblok gua jelek tanda tangan gua jelek
14822,14822,14822,27ad4c813396a959e965fe56f62cd912,Sowerny mati,0,en,en,en,en,0,0,0,0,0,sowerni mati,sowerny mati,sowerni mati
14825,14825,14825,f81df3b4a5842f1fad6e80b3a22394ea,No hot shower and hair dryer. Quite inconvenient,1,en,en,en,en,0,0,0,0,0,hot shower hair dryer quit inconveni,no hot shower and hair dryer quite inconvenient,hot shower hair dryer quit inconveni


In [6]:
for i in mantab['review_text']:
    print(i)

having great services and strategic locations near culinary spots.
the room was comfort, big, and the facilities was good. strategic place in the center of city
clean romm and new building . 
nice room, love it, good food and decdnt price at room service, good breakfast
Although we only stayed 1 night but overall we were satisfied with the facilities and service at thi
dirty and channel tv is locally 

Great place with affordable price
wifi n breakfast definitely need improvement
no shower
exellent experience stay,hotel provide comfortable feel for any guest with drink welcome, friendly s
The hotel is old but we'll maintained. The hotel has lift/elevator but too dark in the lobby and the corridor. 
kamar sempit, air shower macet.
awful. the charged motorcycle parking fee

one of my itinerary was iftar and shalat at masjid keraton yogya. where it close to airy eco malioboro.
location near from benteng vredeburg, keraton, taman pintar, titik nol yogya, and malioboro.
staff was friendly. 

In [7]:
for i in mantab['langdetect']:
    print(i)

en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
Unknown
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
Unknown
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
Unknown
en
en
en
en
en
en
en
en
Unknown
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
Unknown
en
en
en
en
en
en
en
en
en
en
en
en
Unknown
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
Unknown
en
en
en
en
en
en
Unknown
en
Unknown
en
Unknown
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
Unknown
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
Unknown
en
en
en
en
e

In [10]:
train['langdetect'].value_counts()

id         14402
en           433
Unknown       21
Name: langdetect, dtype: int64

In [ ]:
pip -upgrade P